In [40]:
import torch
from torch import nn
from d2l import torch as d2l
from torchvision import transforms

## 实现互相关运算

In [41]:
# K是核矩阵
def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    # 初始化Y，用来存结果
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    # 遍历Y
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
           Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [42]:
X = torch.arange(9, dtype=torch.float32).reshape((3, 3))
K = torch.arange(4, dtype=torch.float32).reshape((2, 2))
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 实现二维卷积层

In [43]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

## 卷积层的一个简单应用：检测图像中不同颜色的边缘

In [44]:
trans = transforms.ToPILImage()

In [45]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [46]:
trans(X).show()

In [47]:
K = torch.tensor([[1.0, -1.0]])

In [48]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [49]:
trans(Y).show()

## 学习由X生成Y的卷积核

In [50]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))  # (批量大小, 通道数, 高度, 宽度)
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f"batch {i + 1}, loss {l.sum():.3f}")

batch 2, loss 4.090
batch 4, loss 0.955
batch 6, loss 0.270
batch 8, loss 0.090
batch 10, loss 0.034


In [51]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9728, -1.0093]])